![title](xyz.png)

In [26]:
!pip install shapash

In [27]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from shapash.explainer.smart_explainer import SmartExplainer 

# Loading Dataset


In [28]:
 df = pd.read_csv("winequality-red.csv")
 y = df['quality']
 X = df.drop(['quality'], axis = 1)


In [29]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


# Splitting Dataset into Train And Test

In [30]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)
 

# Training The Model

In [31]:
model = RandomForestRegressor(max_depth=5, random_state=42, n_estimators=12)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=5, n_estimators=12, random_state=42)

# Prediction

In [32]:
y_pred = pd.DataFrame(model.predict(X_test),columns=['pred'], index=X_test.index)
y_pred.head()

,pred
803,5.326713
124,5.097386
350,5.442830
682,5.132046
1326,5.786685


# Understand Model using Shapash 
In this section, we use the SmartExplainer Object from shapash.
- It allows users to understand how the model works with the specified data. 
- This object must be used only for data mining step. Shapash provides another object for deployment.
- In this tutorial, we are not exploring possibilites of the SmartExplainer but others will. (see other tutorials)

In [33]:
xpl = SmartExplainer()
xpl.compile(x=X_test, model=model, y_pred=y_pred)

Backend: Shap TreeExplainer


## Start Web App

In [34]:
app = xpl.run_app(title_story='Red_Wine_dataset')

Dash is running on http://0.0.0.0:8050/



C:\Users\shubham\anaconda3\envs\machine_learning\lib\site-packages\shapash\webapp\smart_app.py:306: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://MSI:8050/


Dash is running on http://0.0.0.0:8050/



INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app 'shapash.webapp.smart_app' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://192.168.29.92:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:47] "GET /assets/material-icons.css?m=1642254783.7509344 HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:47] "GET /assets/style.css?m=1642254783.7549238 HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:47] "GET /assets/jquery.js?m=1642254783.7499368 HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:47] "GET /assets/main.js?m=1642254783.7509344 HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:48] "GET /_dash-dependencies HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:48] "GET /_dash-layout HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:48] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Jan/2022 21:07:48] "POST /_dash-update-component HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 -

## Stop the web app

In [35]:
app.kill()

# Export Local Explaination in DataFrame

In [36]:
summary_df= xpl.to_pandas(
    max_contrib=3, # Number Max of features to show in summary
)

In [37]:
summary_df.head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
803,5.326713,alcohol,9.6,-0.326036,sulphates,0.66,0.096204,volatile acidity,0.56,-0.05707
124,5.097386,alcohol,9.5,-0.265643,sulphates,0.48,-0.166499,total sulfur dioxide,102.0,-0.065542
350,5.442830,alcohol,9.9,-0.307102,sulphates,0.98,0.157862,total sulfur dioxide,34.0,0.113178
682,5.132046,alcohol,9.8,-0.253048,sulphates,0.54,-0.195841,volatile acidity,0.46,-0.025466
1326,5.786685,alcohol,10.6,0.110827,sulphates,0.6,0.048298,total sulfur dioxide,34.0,0.041631


# Save Smart Explainer
You can save the SmartExplainer Object in a pickle file to make new plots later or launch the WebApp again

In [38]:
xpl.save('./xpl.pkl')

# In Production

### Switch from SmartExplainer to SmartPredictor
When you are satisfied by your results and the explainablity given by Shapash, you can use the SmartPredictor object for deployment. 
- In this section, we learn how to easily switch from SmartExplainer to a SmartPredictor.
- SmartPredictor allows you to make predictions, detail and summarize contributions on new data automatically.
- It only keeps the attributes needed for deployment to be lighter than the SmartExplainer object. 
- SmartPredictor performs additional consistency checks before deployment.
- SmartPredictor allows you to configure the way of summary to suit your use cases.
- It can be used with API or in batch mode.

In [39]:
predictor = xpl.to_smartpredictor()

#### Save your SmartPredictor in Pickle File

In [40]:
predictor.save('./predictor.pkl')

#### Load SmartPredictor

In [41]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

# Make prediction

#### Add Data
The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [44]:
predictor_load.add_input(x=X, ypred=y)

#### Prediction

In [45]:
predictor_load.data["ypred"].head()

,quality
0,5
1,5
2,5
3,6
4,5


## Detail Explainability about Prediction

You can use the method detail_contributions to see the detailed contributions of each of your features for each row of your new dataset.
- For classification problems, it automatically associates contributions with the right predicted label. 
- The predicted label can be computed automatically in the method or you can specify an ypred with add_input method.

In [46]:
detailed_contributions = predictor_load.detail_contributions()
detailed_contributions.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,5,-0.005205,-0.016795,0.006154,0.006258,0.004728,0.003594,0.043806,-0.002750,-0.009844,-0.172608,-0.247921
1,5,0.004325,-0.114696,0.001380,0.007085,-0.044455,-0.021742,-0.053328,-0.002561,0.013933,0.102693,-0.288787
2,5,0.004216,-0.077735,-0.000275,0.005243,-0.010588,0.005058,-0.033811,0.000098,0.013222,0.112628,-0.328322
3,6,-0.015129,0.303148,-0.020723,0.023581,-0.014436,0.005790,-0.021160,-0.003865,0.014717,-0.079952,-0.261541
4,5,-0.005205,-0.016795,0.006154,0.006258,0.004728,0.003594,0.043806,-0.002750,-0.009844,-0.172608,-0.247921
